In [1]:
using System;
using System.Collections.Concurrent;
using System.Threading;

public class Router
{
    private readonly ConcurrentDictionary<int, BlockingCollection<Message>> commandQueues = new ConcurrentDictionary<int, BlockingCollection<Message>>();
    private readonly CancellationTokenSource cancellationTokenSource = new CancellationTokenSource();

    public void AddCommand(int commandId)
    {
        commandQueues[commandId] = new BlockingCollection<Message>();
    }

    public void RemoveCommand(int commandId)
    {
        if (commandQueues.TryRemove(commandId, out var queue))
        {
            queue.CompleteAdding();
        }
    }

    public void SendMessage(Message message)
    {
        if (commandQueues.TryGetValue(message.CommandId, out var queue))
        {
            queue.Add(message);
        }
        else
        {
            Console.WriteLine($"Сообщение для несуществующей команды: {message.CommandId}");
        }
    }

    public void Execute()
    {
        while (!cancellationTokenSource.IsCancellationRequested)
        {
            foreach (var pair in commandQueues)
            {
                Message message;
                if (pair.Value.TryTake(out message))
                {
                    Console.WriteLine($"Роутер: Отправлено сообщение команде {message.CommandId}: {message.Data}");
                }
            }

            Thread.Sleep(100);
        }
    }

    public void Stop()
    {
        cancellationTokenSource.Cancel();
    }
}

public class LongCommand
{
    public int Id { get; }

    public LongCommand(int id)
    {
        Id = id;
    }

    public void Execute()
    {
        // длительная операция
        for (int i = 0; i < 10; i++)
        {
            Thread.Sleep(1000);
            Console.WriteLine($"Команда {Id}: Выполнено {i + 1} из 10 шагов");
        }
    }
}

public class Message
{
    public int CommandId { get; set; }
    public string Data { get; set; }
}

In [2]:
Router router = new Router();

        LongCommand command1 = new LongCommand(1);
        LongCommand command2 = new LongCommand(2);

        router.AddCommand(command1.Id);
        router.AddCommand(command2.Id);

        Thread routerThread = new Thread(router.Execute);
        Thread command1Thread = new Thread(command1.Execute);
        Thread command2Thread = new Thread(command2.Execute);

        routerThread.Start();
        command1Thread.Start();
        command2Thread.Start();

        router.SendMessage(new Message { CommandId = command1.Id, Data = "Сообщение 1 для команды 1" });
        router.SendMessage(new Message { CommandId = command2.Id, Data = "Сообщение 1 для команды 2" });
        router.SendMessage(new Message { CommandId = command1.Id, Data = "Сообщение 2 для команды 1" });
        router.SendMessage(new Message { CommandId = 3, Data = "Сообщение для несуществующей команды" });

        Thread.Sleep(5000);

        router.RemoveCommand(command1.Id);
        router.RemoveCommand(command2.Id);

        router.Stop();

        command1Thread.Join();
        command2Thread.Join();
        routerThread.Join();

Сообщение для несуществующей команды: 3
Роутер: Отправлено сообщение команде 1: Сообщение 1 для команды 1
Роутер: Отправлено сообщение команде 2: Сообщение 1 для команды 2
Роутер: Отправлено сообщение команде 1: Сообщение 2 для команды 1
Команда 1: Выполнено 1 из 10 шагов
Команда 2: Выполнено 1 из 10 шагов
Команда 2: Выполнено 2 из 10 шагов
Команда 1: Выполнено 2 из 10 шагов
Команда 2: Выполнено 3 из 10 шагов
Команда 1: Выполнено 3 из 10 шагов
Команда 1: Выполнено 4 из 10 шагов
Команда 2: Выполнено 4 из 10 шагов
Команда 2: Выполнено 5 из 10 шагов
Команда 1: Выполнено 5 из 10 шагов
Команда 1: Выполнено 6 из 10 шагов
Команда 2: Выполнено 6 из 10 шагов
Команда 1: Выполнено 7 из 10 шагов
Команда 2: Выполнено 7 из 10 шагов
Команда 1: Выполнено 8 из 10 шагов
Команда 2: Выполнено 8 из 10 шагов
Команда 1: Выполнено 9 из 10 шагов
Команда 2: Выполнено 9 из 10 шагов
Команда 1: Выполнено 10 из 10 шагов
Команда 2: Выполнено 10 из 10 шагов
